In [1]:
!pip install lime

     |████████████████████████████████| 275 kB 5.1 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=3398fafc5c2484ca3575014afefb6d1b31b5c5cc262b481fc01aaf7f57cbc1dc
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lime.lime_tabular import LimeTabularExplainer
from matplotlib import pyplot as plt
from google.colab import files
import pandas as pd
import json
import re
import ast
import numpy as np

In [3]:
def process_spectra(file_obj, file_name):
    file_string = str(file_obj[file_name])

    return file_string.split("\\n")

In [4]:
def process_matrix(file_obj, file_name):
    file_string = str(file_obj[file_name])

    result = file_string.split("\\n")
    result[0] = result[0].replace("b'", "")

    for i in range(len(result)):
        result[i] = result[i].split(" ")
        result[i] = [int(x) if x.isdigit() else x for x in result[i]]
        lastel = result[i][-1]
        if lastel == "+":
            result[i][-1] = 1
        elif lastel == "-":
            result[i][-1] = 0
    
    return result

In [5]:
def make_line_in_txt_lime(exp_instance):
    exp_in_list = exp_instance.as_list()
    new_list = []

    for i in exp_in_list:
        if ("<=" in i[0]):
            string_split = " <= "
        elif (">=" in i[0]):
            string_split = " >= "
        elif ("<" in i[0]):
            string_split = " < "
        elif (">" in i[0]):
            string_split = " > "
        feature = i[0].split(string_split)
        index = int(feature[0])
        feature[0] = list(df.columns.values)[index]
        new_list.append(";".join([feature[0], str(i[1])]))
    
    return new_list

In [25]:
def write_output_file(filename, obj, is_reverse=False):
    with open(filename, 'w') as file:
        sorted_dict = sorted(obj.items(), key=lambda x: x[1], reverse=is_reverse)
        for i in sorted_dict:
            file.write(";".join([i[0], str(i[1])]) + "\n")

In [6]:
spectra_file = files.upload()
matrix_file = files.upload()

Saving spectra to spectra


Saving matrix to matrix


In [7]:
spectra_list = process_spectra(spectra_file, list(spectra_file.keys())[0])
spectra_list[-1] = "Pass/Fail"
spectra_list[0] = spectra_list[0].replace("b'", "")

In [8]:
matrix_list = process_matrix(matrix_file, list(matrix_file.keys())[0])

In [9]:
matrix_list.pop()

["'"]

In [10]:
df = pd.DataFrame(data=matrix_list, columns=spectra_list)

In [11]:
df

,org.apache.commons.math3.util.MathArrays$1#674,org.apache.commons.math3.util.MathArrays$1#678,org.apache.commons.math3.util.MathArrays$1#680,org.apache.commons.math3.util.MathArrays$1#681,org.apache.commons.math3.util.MathArrays$1#683,org.apache.commons.math3.util.MathArrays$1#684,org.apache.commons.math3.util.MathArrays$1#687,org.apache.commons.math3.util.MathArrays$1#689,org.apache.commons.math3.util.MathArrays#51,org.apache.commons.math3.util.MathArrays#88,org.apache.commons.math3.util.MathArrays#89,org.apache.commons.math3.util.MathArrays#92,org.apache.commons.math3.util.MathArrays#93,org.apache.commons.math3.util.MathArrays#94,org.apache.commons.math3.util.MathArrays#96,org.apache.commons.math3.util.MathArrays#110,org.apache.commons.math3.util.MathArrays#111,org.apache.commons.math3.util.MathArrays#114,org.apache.commons.math3.util.MathArrays#115,org.apache.commons.math3.util.MathArrays#116,org.apache.commons.math3.util.MathArrays#118,org.apache.commons.math3.util.MathArrays#132,org.apache.commons.math3.util.MathArrays#133,org.apache.commons.math3.util.MathArrays#136,org.apache.commons.math3.util.MathArrays#137,org.apache.commons.math3.util.MathArrays#138,org.apache.commons.math3.util.MathArrays#140,org.apache.commons.math3.util.MathArrays#154,org.apache.commons.math3.util.MathArrays#155,org.apache.commons.math3.util.MathArrays#158,org.apache.commons.math3.util.MathArrays#159,org.apache.commons.math3.util.MathArrays#160,org.apache.commons.math3.util.MathArrays#162,org.apache.commons.math3.util.MathArrays#173,org.apache.commons.math3.util.MathArrays#174,org.apache.commons.math3.util.MathArrays#175,org.apache.commons.math3.util.MathArrays#177,org.apache.commons.math3.util.MathArrays#188,org.apache.commons.math3.util.MathArrays#189,org.apache.commons.math3.util.MathArrays#190,...,org.apache.commons.math3.random.RandomDataImpl#127,org.apache.commons.math3.random.RandomDataImpl#128,org.apache.commons.math3.random.RandomDataImpl#136,org.apache.commons.math3.random.RandomDataImpl#156,org.apache.commons.math3.random.RandomDataImpl#161,org.apache.commons.math3.random.RandomDataImpl#166,org.apache.commons.math3.random.RandomDataImpl#186,org.apache.commons.math3.random.RandomDataImpl#191,org.apache.commons.math3.random.RandomDataImpl#196,org.apache.commons.math3.random.RandomDataImpl#213,org.apache.commons.math3.random.RandomDataImpl#218,org.apache.commons.math3.random.RandomDataImpl#233,org.apache.commons.math3.random.RandomDataImpl#248,org.apache.commons.math3.random.RandomDataImpl#265,org.apache.commons.math3.random.RandomDataImpl#279,org.apache.commons.math3.random.RandomDataImpl#293,org.apache.commons.math3.random.RandomDataImpl#307,org.apache.commons.math3.random.RandomDataImpl#320,org.apache.commons.math3.random.RandomDataImpl#336,org.apache.commons.math3.random.RandomDataImpl#363,org.apache.commons.math3.random.RandomDataImpl#383,org.apache.commons.math3.random.RandomDataImpl#401,org.apache.commons.math3.random.RandomDataImpl#415,org.apache.commons.math3.random.RandomDataImpl#431,org.apache.commons.math3.random.RandomDataImpl#447,org.apache.commons.math3.random.RandomDataImpl#461,org.apache.commons.math3.random.RandomDataImpl#462,org.apache.commons.math3.random.RandomDataImpl#472,org.apache.commons.math3.random.RandomDataImpl#473,org.apache.commons.math3.random.RandomDataImpl#485,org.apache.commons.math3.random.RandomDataImpl#486,org.apache.commons.math3.random.RandomDataImpl#493,org.apache.commons.math3.random.RandomDataImpl#494,org.apache.commons.math3.random.RandomDataImpl#518,org.apache.commons.math3.random.RandomDataImpl#519,org.apache.commons.math3.random.RandomDataImpl#532,org.apache.commons.math3.random.RandomDataImpl#550,org.apache.commons.math3.random.RandomDataImpl#565,org.apache.commons.math3.random.RandomDataImpl#581,Pass/Fail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0

In [12]:
x = df[spectra_list[:len(spectra_list)-1]]
y = df["Pass/Fail"]

In [13]:
fails = []
passes = []
i = 0
for result in y:
    if result == 0:
        fails.append(i)
    else:
        passes.append(i)
    i += 1 

In [14]:
if len(fails) == 1:
    df = df.append(df.iloc[fails[0]])
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [15]:
if (df.shape[0] < 6):
    df = df.append(df)
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [17]:
model = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', random_state=1)

In [18]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [20]:
lime_to_txt = {}

explainer = LimeTabularExplainer(training_data= np.array(x_train), mode="classification", feature_selection="auto")

for i in fails:
    exp_instance = explainer.explain_instance(x.iloc[i], predict_fn=model.predict_proba)
    lime_to_txt[x.iloc[i].name] = []
    lime_to_txt[x.iloc[i].name] += make_line_in_txt_lime(exp_instance)    

In [21]:
lime_to_txt

{18: ['org.apache.commons.math3.distribution.DiscreteDistribution#189;-0.049666255863130934',
  'org.apache.commons.math3.distribution.DiscreteDistribution#69;-0.04339179579174787',
  'org.apache.commons.math3.distribution.DiscreteDistribution#187;-0.039688899886174876',
  'org.apache.commons.math3.distribution.DiscreteDistribution#190;-0.035049981873795294',
  'org.apache.commons.math3.distribution.DiscreteDistribution#161;-0.03460524282922955',
  'org.apache.commons.math3.random.BitsStreamGenerator#92;-0.030422524133544255',
  'org.apache.commons.math3.random.Well19937c#98;-0.02952056888533977',
  'org.apache.commons.math3.distribution.DiscreteDistribution#158;-0.029342302774549286',
  'org.apache.commons.math3.distribution.DiscreteDistribution#162;-0.02782847401066114',
  'org.apache.commons.math3.random.BitsStreamGenerator#91;-0.02383727130538319']}

In [22]:
lime_lines_output_val = {}

for i in lime_to_txt:
    for index, value in enumerate(lime_to_txt[i]):
        feature_and_value = value.split(";")
        feature = feature_and_value[0]
        lime_val =  float(feature_and_value[1])
        if feature not in lime_lines_output_val:
            lime_lines_output_val[feature] = []
        lime_lines_output_val[feature].append(lime_val)

In [24]:
lime_means = {}
lime_max = {}
lime_min = {}

for i in lime_lines_output_val:
    if i not in lime_means:
        lime_means[i] = []
    if i not in lime_min:
        lime_min[i] = lime_lines_output_val[i][0]
    if i not in lime_max:
        lime_max[i] = lime_lines_output_val[i][0]
    for j in lime_lines_output_val[i]:
        lime_means[i].append(j)
        if abs(lime_max[i]) < abs(j):
            lime_max[i] = j
        if abs(lime_min[i]) > abs(j):
            lime_min[i] = j

for i in lime_means:
    lime_means[i] = sum(lime_means[i]) / len(lime_means[i])

In [26]:
with open("Math-8_lime_results.txt", "w") as file:
    for i in lime_to_txt:
        file.write(str(i) + "\n")
        for j in range(len(lime_to_txt[i])):
            file.write(lime_to_txt[i][j] + "\n")

In [27]:
write_output_file('Math-8_shap_mean.txt', lime_means)
write_output_file('Math-8_shap_min.txt', lime_min)
write_output_file('Math-8_shap_max.txt', lime_max)